In [1]:
from __future__ import annotations
from io import BytesIO

import numpy as np
import oxbow as ox
import pandas as pd
import polars as pl
import pyarrow as pa
import pysam

In [58]:
# This dictionary maps the VCF-derived input types to numpy/pandas dtypes
TYPE_MAP = {
    "Integer": "int64",
    "Float": "float64",
    "String": "object",
    "Flag": "bool",
}


def read_vcf_as_pandas(vcf_path: str) -> pd.DataFrame:
    ipc = ox.read_vcf(vcf_path)
    return pa.ipc.open_file(BytesIO(ipc)).read_pandas()


def read_vcf_as_polars(vcf_path: str) -> pl.DataFrame:
    ipc = ox.read_vcf(vcf_path)
    return pl.read_ipc(ipc)


def read_info_schema(vcf_path: str) -> pd.DataFrame:
    """ 
    Read the schema of the INFO column of a VCF.

    This currently uses pysam.

    Parameters
    ----------
    vcf_path : str
        Path to bgzipped vcf, with tabix indexed .tbi file with similar name 
        in same folder.
    
    Returns
    -------
    pd.DataFrame
        Dataframe with [name, number, type] columns

    Notes
    -----
    Possible values for `type` are: "Integer", "Float", "String", "Flag".

    Possible values for `number` are:
        - An integer (e.g. 0, 1, 2, 3, 4, etc.) - for fields where the number
          of values per VCF record is fixed. 0 means the field is a "Flag".
        - A string ("A", "G", "R") - for fields where the number of
          values per VCF record is determined by the number of alts, the total
          number of alleles, or the number of genotypes, respectively.
        - A dot (".") - for fields where the number of values per VCF record
          varies, is unknown, or is unbounded.
    """
    with pysam.VariantFile(vcf_path) as f:
        return pd.DataFrame(
            [(obj.name, obj.number, obj.type, obj.description) for obj in f.header.info.values()],
            columns=["name", "number", "type", "description"],
        )


def read_format_schema(vcf_path: str) -> pd.DataFrame:
    """ 
    Read the schema of the sample genotypes of a VCF.

    This currently uses pysam.

    Parameters
    ----------
    vcf_path : str
        Path to bgzipped vcf, with tabix indexed .tbi file with similar name 
        in same folder.
    
    Returns
    -------
    pd.DataFrame
        Dataframe with [name, number, type] columns

    Notes
    -----
    Possible values for `type` are: "Integer", "Float", "String", "Flag".

    Possible values for `number` are:
        - An integer (e.g. 0, 1, 2, 3, 4, etc.) - for fields where the number
          of values per VCF record is fixed. 0 means the field is a "Flag".
        - A string ("A", "G", "R") - for fields where the number of
          values per VCF record is determined by the number of alts, the total
          number of alleles, or the number of genotypes, respectively.
        - A dot (".") - for fields where the number of values per VCF record
          varies, is unknown, or is unbounded.
    """
    with pysam.VariantFile(vcf_path) as f:
        return pd.DataFrame(
            [(obj.name, obj.number, obj.type, obj.description) for obj in f.header.formats.values()],
            columns=["name", "number", "type", "description"],
        )

In [142]:
def read_vcf_as_pandas(
    path: str,
    query: tuple | None = None,
    info_fields: list[str] | None = None,
    sample_fields: list[str] | None = None,
    samples: list[str] | None = None,
) -> pd.DataFrame:
    
    f = pysam.VariantFile(path)
    info_schema = read_info_schema(path)
    sample_schema = read_format_schema(path)
    info_arity = info_schema.set_index("name")["number"]
    sample_arity = sample_schema.set_index("name")["number"]

    if info_fields is None:
        info_fields = list(info_schema["name"])
    if sample_fields is None:
        sample_fields = list(sample_schema["name"])
    if samples is None:
        samples = list(f.header.samples)
    
    if query is None:
        record_iter = f
    else:
        record_iter = f.fetch(*query)

    record_dicts = []
    for record in record_iter:
        record_dict = {
            "chrom": record.chrom,
            "pos": record.pos,
            "id": record.id,
            "ref": record.ref,
            "alts": np.array(record.alts),
            "qual": record.qual,
            "filters": np.array(record.filter.keys()),
        }
        
        for field in info_fields:
            if field in record.info:
                record_dict[field] = record.info[field]
                if info_arity[field] not in {0, 1}:
                    record_dict[field] = np.array(record_dict[field])
            
        for sample in samples:
            for field in sample_fields:
                if field in record.samples[sample]:
                    record_dict[f"{sample}.{field}"] = record.samples[sample][field]
                    if sample_arity[field] not in {0, 1}:
                        record_dict[f"{sample}.{field}"] = np.array(record_dict[f"{sample}.{field}"])
                if field == "GT":
                    record_dict[f"{sample}.{field}"] = np.array(record_dict[f"{sample}.{field}"])
                    record_dict[f"{sample}.phased"] = record.samples[sample].phased
        record_dicts.append(record_dict)

    df = pd.DataFrame.from_records(record_dicts)

    columns = (
        ["chrom", "pos", "id", "ref", "alts", "qual", "filters"] 
        + info_fields 
        + [f"{sample}.{field}" for sample in samples 
           for field in [*sample_fields, f"phased"]]
    )

    for col in columns:
        if col not in df.columns:
            df[col] = pd.NA

    return df[columns].fillna(np.nan)


In [122]:
df = read_vcf_as_pandas("DRR259112.ref.snpeff.vcf", info_fields=["DP"], sample_fields=["GT"])

### Find the longest alt

In [143]:
df2 = df.explode("alts")
df2.loc[df2['alts'].apply(len).idxmax()]

chrom                  NC_045512.2
pos                          21897
id                            <NA>
ref                              C
alts                CTATATTAGAGTAG
qual                   6963.600098
filters                     [PASS]
DP                             552
DRR259112.GT                [0, 1]
DRR259112.phased             False
Name: 27, dtype: object

In [158]:
info_fields = ["LOF"]
df1 = read_vcf_as_pandas("DRR259112.ref.snpeff.vcf", info_fields=info_fields, sample_fields=["GT"])
df2 = read_vcf_as_pandas("DRR259113.ref.snpeff.vcf.gz", info_fields=info_fields, sample_fields=["GT"])

In [165]:
read_info_schema("s3://sra-pub-sars-cov2/vcf/DRR259112/DRR259112.ref.snpeff.vcf")

,name,number,type,description
0,DP,1,Integer,Approximate read depth; some reads may have be...
1,AC,A,Integer,"Allele count in genotypes, for each ALT allele..."
2,AF,A,Float,"Allele Frequency, for each ALT allele, in the ..."
3,AN,1,Integer,Total number of alleles in called genotypes
4,BaseQRankSum,1,Float,Z-score from Wilcoxon rank sum test of Alt Vs....
5,ExcessHet,1,Float,Phred-scaled p-value for exact test of excess ...
6,FS,1,Float,Phred-scaled p-value using Fisher's exact test...
7,InbreedingCoeff,1,Float,Inbreeding coefficient as estimated from the g...
8,MLEAC,A,Integer,Maximum likelihood expectation (MLE) for the a...
9,MLEAF,A,Float,Maximum likelihood expectation (MLE) for the a...


### Merge VCFs

In [161]:
pd.merge(
    df1.drop(columns=["qual", "id", "alts", "filters"]),
    df2.drop(columns=["qual", "id", "alts", "filters"]),
    how="outer",
    on=["chrom", "pos", "ref"],
    suffixes=(".DRR259112", ".DRR259113")
)

,chrom,pos,ref,LOF.DRR259112,DRR259112.GT,DRR259112.phased,LOF.DRR259113,DRR259113.GT,DRR259113.phased
0,NC_045512.2,241,C,NaN,"[0, 1]",False,NaN,NaN,NaN
1,NC_045512.2,316,TTT,[(leader|leader-gene|1|1.00)],"[0, 1]",False,NaN,NaN,NaN
2,NC_045512.2,2662,C,NaN,"[1, 1]",False,NaN,"[1, 1]",False
3,NC_045512.2,3371,G,NaN,"[0, 1]",False,NaN,NaN,NaN
4,NC_045512.2,4290,C,NaN,"[0, 1]",False,NaN,NaN,NaN
5,NC_045512.2,4456,C,NaN,"[1, 1]",False,NaN,NaN,NaN
6,NC_045512.2,4890,C,NaN,"[0, 1]",False,NaN,NaN,NaN
7,NC_045512.2,5497,C,NaN,"[0, 1]",False,NaN,NaN,NaN
8,NC_045512.2,6522,T,NaN,"[0, 1]",False,NaN,NaN,NaN
9,NC_045512.2,6941,C,NaN,"[0, 1]",False,NaN,NaN,NaN
